In [0]:
!pip install imageio

In [0]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os, tarfile
import imageio
import tensorflow as tf
from scipy.io import loadmat
# from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

tf.test.gpu_device_name()

''

In [0]:
fmnist_TRAIN_URL = 'https://www.dropbox.com/s/ynakalalovlqi5j/fashion-mnist_train.csv?dl=1'
fminst_TEST_URL = 'https://www.dropbox.com/s/ynakalalovlqi5j/fashion-mnist_train.csv?dl=1'

# Utility functions

#### ToDos
- Create a function to fetch data from a url.
- Check if it is already downloaded.
- Check if the file is csv or tar gz etc.
- Add cross-validation code to be able to use sklearn cross_val_score function to quickly evaluate the performance.

In [0]:
def fetch_data(URL, DOWNLOAD_FOLDER, DOWNLOAD_FILE):
  if not os.path.isdir(DOWNLOAD_FOLDER):
   os.makedirs(DOWNLOAD_FOLDER)
  
  if not os.path.isfile(DOWNLOAD_FOLDER+DOWNLOAD_FILE):
    print('Beginning file download...')
    urllib.request.urlretrieve(URL, DOWNLOAD_FOLDER+DOWNLOAD_FILE)
    print('Done.')
  

In [0]:
def split_train_test(XY, n_splits=1, test_size=0.2, random_state=42):
    split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(XY[0], XY[1]):
        X_train, Y_train = XY[0][train_index,:], XY[1][train_index]
        X_test, Y_test = XY[0][test_index,:], XY[1][test_index]
        
    return X_train, Y_train, X_test, Y_test

In [0]:
def get_fmnist_data(url, download_folder, download_file, n_samples, split_data=False):
    
    fetch_data(url, download_folder, download_file)
    XY = np.loadtxt(download_folder+download_file, dtype=np.float32, delimiter=',', skiprows=1)
    X = XY[:,1:]
    Y = XY[:,0]
    
    X = np.array(X) / 255.0
    
    #X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)

    if split_data:
        return split_train_test((X_new, Y), n_splits=1, test_size=0.2, random_state=42)

    return X_train, np.expand_dims(Y_train,1), X_test, np.expand_dims(Y_test,1)

In [0]:
labels = np.arange(0,10,1)
print(labels[1:])

[1 2 3 4 5 6 7 8 9]


In [0]:
def one_hot_encoder(label):
    encoder = OneHotEncoder(dtype=np.float32)
    label_1hot = encoder.fit_transform(label.reshape(-1,1))
    print('The labels are: {}'.format(np.unique(label)))
    return label_1hot

# Load data

In [0]:
root_folder = 'drive/app/fer/'
# root_folder = 'D:/dev/data/'

In [0]:
X_train, Y_train, X_test, Y_test = get_fer_data(FER_URL, root_folder, 'fer2013.csv',
                                                split_data=False)

print("Train: [{}, {}], Test: [{}, {}]".format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

Train: [(28709, 2304), (28709, 1)], Test: [(7178, 2304), (7178, 1)]


In [0]:
Y_train_1hot = one_hot_encoder(Y_train).toarray().view(np.float32)
Y_test_1hot = one_hot_encoder(Y_test).toarray().view(np.float32)
# print(Y_train_1hot[0:2])
# print(type(Y_train_1hot))

The labels are: [0 1 2 3 4 5 6]
The labels are: [0 1 2 3 4 5 6]


In [0]:
plt.imshow(X_train[600,:,:,:])
plt.title(Y_train[600,0])

IndexError: ignored

In [0]:
class CNN(object):
    def __init__(self, width, height, n_channels, n_classes):

        self.width = width
        self.height = height
        self.channels = n_channels
        self.classes = n_classes
        self.X = tf.placeholder(tf.float32, shape=(None, width*height*n_channels), name='X')
        self.Y = tf.placeholder(tf.float32, shape=(None, n_classes), name='Y')
        self.keep_prob = tf.placeholder(tf.float32)
        self.set_pipeline()
        self.session = tf.InteractiveSession()
    
    def forward(self, X):
        
        self.input = tf.reshape(X, [-1, self.width, self.height, self.channels])
        conv_11 = tf.layers.conv2d(self.input, filters=32, kernel_size=[5,5], padding='SAME', activation=tf.nn.relu)
        drop_11 = tf.layers.dropout(conv_11, self.keep_prob)
        conv_12 = tf.layers.conv2d(conv_11, filters=64, kernel_size=[5,5], padding='SAME', activation=tf.nn.relu)
        pool_11 = tf.layers.max_pooling2d(conv_12, pool_size=[2,2], strides=2)

        conv_21 = tf.layers.conv2d(pool_11, filters=128, kernel_size=[5,5], padding='SAME', activation=tf.nn.relu)
        drop_21 = tf.layers.dropout(conv_21, self.keep_prob)
        conv_22 = tf.layers.conv2d(conv_21, filters=256, kernel_size=[5,5], padding='SAME', activation=tf.nn.relu)
        pool_21 = tf.layers.max_pooling2d(conv_22, pool_size=[2,2], strides=2)

        flat_3 = tf.layers.flatten(pool_21)

        fc_4 = tf.layers.dense(inputs=flat_3, units=1024, activation=tf.nn.relu)
        drop_5 = tf.layers.dropout(fc_4, self.keep_prob)
        fc_5 = tf.layers.dense(inputs=fc_4, units=self.classes)
        return fc_5

    def set_pipeline(self):

        Y_pred = self.forward(self.X)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=Y_pred))
        self.optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001, decay=0.95, momentum=0.9).minimize(self.cost)
        self.predictions = tf.argmax(Y_pred, axis=1)


    def fit(self, X, Y, MAX_ITER, n_batches, batch_size, print_time=None):

        Y_1hot = one_hot_encoder(Y).toarray().view(np.float32)
        
        init = tf.global_variables_initializer()
        self.session.run(init)
        for iter in range(MAX_ITER):
            X_shuffled, Y_1hot_shuffled, Y_shuffled = shuffle(X, Y_1hot, Y)
            for ibatch in range(n_batches):
                X_batch = X_shuffled[ibatch*batch_size:(ibatch+1)*batch_size,:]
                Y_1hot_batch = Y_1hot_shuffled[ibatch*batch_size:(ibatch+1)*batch_size,:]
                Y_batch = Y_shuffled[ibatch*batch_size:(ibatch+1)*batch_size,:]

                #print(X_batch.dtype, Y_batch.dtype)

                # Training
                train_cost = self.session.run(self.cost, feed_dict={self.X:X_batch, self.Y:Y_1hot_batch})
                self.session.run(self.optimizer, feed_dict={self.X:X_batch, self.Y:Y_1hot_batch, self.keep_prob:0.5})

                if print_time==None:
                    print_time = n_batches//2

                if ibatch%print_time == 0:
                    print('Epoch:{}, Batch:{}, Train loss:{}'.format(iter, ibatch, train_cost))


    def predict(self, X, Y, n_batches, batch_size):

        correct = 0
        test_cost = 0
        total = 0
        Y_1hot = one_hot_encoder(Y).toarray().view(np.float32)
        for ibatch in range(n_batches):
            X_batch = X[ibatch*batch_size:(ibatch+1)*batch_size,:]
            Y_1hot_batch = Y_1hot[ibatch*batch_size:(ibatch+1)*batch_size,:]
            Y_batch = Y[ibatch*batch_size:(ibatch+1)*batch_size,:]

            test_cost += self.session.run(self.cost, feed_dict={self.X:X_batch,
                                                           self.Y:Y_1hot_batch, 
                                                           self.keep_prob:1.0})

            test_predictions = np.expand_dims(self.session.run(self.predictions, feed_dict={self.X:X_batch}), 1)

            #Y_batch = Y_test[ibatch*batch_size:(ibatch+1)*batch_size,:]
            correct += np.sum(test_predictions==Y_batch)
            total += len(Y_batch)

        print('Test loss:{}, Accuracy:{}%'.format(test_cost/n_batches, np.round(correct*100/total, 2)))


In [0]:
WIDTH = 28
HEIGHT = 28
N_CHANNELS = 1
N_CLASSES = 10
BATCH_SIZE = 64
MAX_ITER = 10
N_BATCHES = X_train.shape[0]//BATCH_SIZE
PRINT_TIME = N_BATCHES//2
TEST_N_BATCHES = X_test.shape[0]//BATCH_SIZE

In [0]:
ann = CNN(WIDTH, HEIGHT, N_CHANNELS, N_CLASSES)

In [0]:
ann.fit(X_train, Y_train, MAX_ITER, N_BATCHES, BATCH_SIZE, PRINT_TIME)

The labels are: [0 1 2 3 4 5 6]
Epoch:0, Batch:0, Train loss:1.9411547183990479
Epoch:0, Batch:224, Train loss:1.8334064483642578
Epoch:1, Batch:0, Train loss:1.4204095602035522


In [0]:
ann.predict(X_test, Y_test, TEST_N_BATCHES, BATCH_SIZE)

The labels are: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Test loss:0.5225323887881387, Accuracy:86.08%


In [0]:
!kill -9 -1